In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
train_instance_num, pixel_num = mnist.train.images.shape
test_instance_num, class_num = mnist.test.labels.shape

In [4]:
x = tf.placeholder('float', shape=[None, pixel_num])
y_answer = tf.placeholder('float', shape=[None, class_num])

In [5]:
x_image = tf.reshape(x, [-1, 28, 28, 1]) # 입력데이터를 원래 이미지의 구조로 재구성한다.
x_image

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

## 필터용 W를 정의

In [6]:
"""
여기에서 초기 가중치를 임의잡음(random noise) 으로 잡은 것은, 초기화 값에 따라서 서로 다른 특징들을 뽑아내길 기대하기 때문으로 볼 수 있다.
"""
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

## 필터용 bias 를 정의

In [7]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## 합성곱 계층을 정의하기

In [8]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

## 맥스 풀링 계층 정의하기

In [9]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

## 첫번째 합성곱 계층 과 풀링 계층을 만들기

In [10]:
W_conv1 = weight_variable([5,5,1,32]) # 5*5 크기의 1채널을 상대하는 32개 의 필터를 생성
b_conv1 = bias_variable([32]) # 32 개의 필터에 들어갈 편향값

In [11]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # relu 로 activation function 을 만들어 Hidden Layer 를 구성
h_pool1 = max_pool_2x2(h_conv1)

## 두번째 합성곱 계층 과 풀링 계층 만들기

In [12]:
W_conv2 = weight_variable([5,5,32,64]) # 5*5 크기의 32 채널 상대, 64개의 필터를 생성
b_conv2 = bias_variable([64]) # 64개의 필터에 들어갈 편향값들
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # 여기서의 인풋은 1단계 Hidden Layer 의 결과값이 들어가게 됨
h_pool2 = max_pool_2x2(h_conv2)

## 소프트맥스 계층에 주입하기 위해서 FC 계층을 만든다

In [13]:
W_fc1 = weight_variable([7*7*64, 1024]) # 7*7 짜리 pool 결과가 64개가 있고, 이 것들을 전부 연결하고, 이러한 뉴런의 개수가 1024 개로 정의
b_fc1 = bias_variable([1024]) # 1024 개의 뉴런의 편향값

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) # 1자 벡터화
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  # 완전 연결 계층 생성

## Dropout 적용

In [14]:
"""
신경망에서 필요한 매개변수 수를 줄이는 것
노드를 삭제하여 입력과 출력 사이의 연결을 제거하는 것
어떤 뉴런을 제거하고, 어떤 뉴런을 유지할 지는 무작위로 결정
뉴런이 제거되거나 그렇지 않을 확률은 코드로 처리하지 않고 텐서플로에게 위임
드롭아웃은 모델이 데이터에 오버피팅 되는 현상을 막아주는 효과
오버피팅이란 입력 데이터의 차원에 비해서 더 많은 매개변수를 가지는 모델에서 자주 발생하는 현상
"""
keep_prob = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## 소프트맥스 계층에 주입

In [15]:
W_fc2 = weight_variable([1024, class_num]) # 1024 개의 뉴런의 결과를 10개의 뉴런으로 압축처리 -> 10개의 클래스로 결정
b_fc2 = bias_variable([class_num]) # 10 개의 클래스의 편향값

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2) # 10 개의 클래스로 결과가 softmax 함수값으로 결정 될 것

## 모델 훈련 및 평가

In [16]:
cross_entropy = - tf.reduce_sum(y_answer*tf.log(y_conv)) # 크로스 엔트로피 방식으로 오차값을 선정
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_answer, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [17]:
sess = tf.Session()

In [18]:
sess.run(tf.global_variables_initializer())
for i in range(1000):
    batch = mnist.train.next_batch(100) # 100 개씩 랜덤하게 픽해서 가져와 학습
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_answer:batch[1], keep_prob:1.0})
        print("step %d, training accuracy : %g" % (i, train_accuracy))
    sess.run(train_step, feed_dict={x:batch[0], y_answer:batch[1], keep_prob:0.5})

print("===최종학습 결과===")
print("test accuracy : %g" % sess.run(accuracy, feed_dict={x:mnist.test.images, y_answer:mnist.test.labels, keep_prob:1.0}))

step 0, training accuracy : 0.06
step 100, training accuracy : 0.9
step 200, training accuracy : 0.93
step 300, training accuracy : 0.96
step 400, training accuracy : 0.92
step 500, training accuracy : 0.94
step 600, training accuracy : 0.96
step 700, training accuracy : 0.92
step 800, training accuracy : 0.9
step 900, training accuracy : 0.95
===최종학습 결과===
test accuracy : 0.9685
